# Outline

In this Notebook we consider graph $\Gamma$ a $n$-necklace labelled by $L=[n]$, $T= \Gamma \setminus e_{1} $ and $D_{T}= \vec{0}$, and provide a implementation of Theorem 4.1.4  to calculate for any cycle $\tau \in C_{L}$ the corresponding $\sigma_{\Gamma} \in \Sigma_{\Gamma}^{d}$ as the list of elements of $\sigma_{\Gamma}(\Gamma)$ and a list of elements of $\sigma_{\Gamma}(T^{'})$ for all $T^{'} \in \mathcal{ST}(\Gamma)$.

Notes:

0) Please run #Functions before going to #Example.

1) The construction here is general and one can enter any n-cycle to obtain the assoicated stability condtion for the $n$ -necklace graph.

2) The orientation of the cycle is incorporated into its implementation here.

# Table of contents

1. [Functions](#s1)
2. [Example](#s2)

# Functions <a name="s1"></a>

We will explain the main functions and leave the subfunctions. See the docstring for a description of specific functions.

In [4]:
from itertools import combinations,permutations
import numpy as np

The next series of functions allow one to compute

$$\Delta^{\tau}_{l_i}:=\delta_{h\left( e_{l_{i}} \right)}-\delta_{t\left(e_{l_{i+1}}\right)} \quad \text{and} \quad \Delta^{\tau}_{l_k:{l_k^{\prime}}}:=\sum_{k \leq i < k^{\prime}} \Delta^{\tau}_{l_i}.$$

in particular $\sigma_{\Gamma}(T_{l_k})=\{\vec{d}_{T_{l_k}}\}$ where $\vec{d}_{T_{l_{1}}}:=D_T$, $\vec{d}_{T_{l_k}}:= D_T+ \Delta_{l_1:l_k}^{\tau}$ for $1 \le k\le n$.

In [7]:
def head(n, prev_tree_edge):
    """
    n= number of vertices in \Gamma
    
    previous element of the cycle, ie tree_edge is j where e_j ={v_j,v_{j+1}} 
    """
    head = np.zeros(n)

    if prev_tree_edge == n:
        head[0] = 1
    else:
        head[prev_tree_edge] = 1

    return head


def tail(n, current_tree_edge):
    """
    n= number of vertices in I_n
    tree_edge is j where e_j ={v_j,v_{j+1}}
    i is previous element of the cycle, ie tree_edge
    """
    tail = np.zeros(n)
    # if current_tree_edge==n:
    #     tail[0]=-1
    tail[current_tree_edge - 1] = -1

    return tail

The next main function is cycle_to_assignments(). For a fixed cycle $\tau \in C_{L}$ the function cycle_to_assignments() returns $A_{\Gamma}$, following Theorem 4.1.4 (genus 1 theorem).

In [5]:
def get_cycles(n):
    """
    Objective: Gets a list of all possible length n cycles (permutations) beginning with 1
    Input: n an integer.
    Returns: a list of all possible length n cycles (permutations) beginning with 1
    """

    lst = list(range(1, n + 1))
    x = permutations(lst, n)
    y = [i for i in x if i[0] == 1]

    #Does:
    # x=permutations([1,2,3,4,5],5)
    # y=[i for i in x if i[0]==1]
    # # print(len(y))
    # for i in y:
    #     print(i)

    return y

In [6]:
#Main to get assingments from cycle
def cycle_to_assignments(cycle):
    """
    Objective: Get the assoicated A_{\Gamma} for the given cycle.
    
    Input:
    cycle: is a length n cycle corresponding to the permutation of [1,...,n]
    
    Returns:
    A list of (i,assignment) where i is the label of the missing edge and
    defines the tree $T= \Gamma \setminus e_i$ for which the assignment is on, and the assignment is $A_{\Gamma}(T)$.
    """

    n = len(cycle)  # cycle=(1,3,4,2)

    ass_0 = np.zeros(n)

    M = [(1, ass_0)]  # Memory

    for i in range(1, len(cycle)):

        current_tree_edge = cycle[i]
        tup = M[-1]  #pick last item of M

        vtail = tail(n, current_tree_edge)
        vhead = head(n, tup[0])
        prev_ass = tup[1]

        current_ass_t = prev_ass + vtail + vhead
        M.append((current_tree_edge, current_ass_t))

    # for i in M:
    #     print(i)

    return M


The final function allows one to determine $\sigma_{\Gamma}(\Gamma)=\{\vec{d}_{l_1},\dots,\vec{d}_{l_n}\}$ and $\vec{d}_{l_k}:= \vec{d}_{T_{l_k}}+ \delta_{h(e_{l_k})}$ for $1 \le k\le n$. 

    


In [ ]:
def tail_plus(n, current_tree_edge):  #for chip adding
    """
    n= number of vertices in I_n
    tree_edge is j where e_j ={v_j,v_{j+1}}
    i is previous element of the cycle, ie tree_edge
    """
    tail = np.zeros(n)
    # if current_tree_edge==n:
    #     tail[0]=-1
    tail[current_tree_edge - 1] = 1

    return tail

# Example <a name="s2"></a>

Given a $n$ cycle we can obtain the tree assignment $A_{\Gamma}$. The output will be a list of $(T,D_{T})$ where $T=\Gamma \setminus e_{i}$ for example in (4, array([ 0.,  1.,  0., -1.]) we have the tree with missing edge $e_{4}$ and divisor  $(0 v_{0},  1v_{1},  0v_{2}, -1v_{3})$.

In [8]:
cycle=(1,4,2,3) #<--- Enter any cycle (begining with 1) here to compute the assoicated stability condition
M=cycle_to_assignments(cycle)
n=len(cycle)
print(M)

# Output:

# [(1, array([0., 0., 0., 0.])),
#  (4, array([ 0.,  1.,  0., -1.])),
#  (2, array([ 1.,  0.,  0., -1.])),
#  (3, array([ 1.,  0.,  0., -1.]))]

[(1, array([0., 0., 0., 0.])), (4, array([ 0.,  1.,  0., -1.])), (2, array([ 1.,  0.,  0., -1.])), (3, array([ 1.,  0.,  0., -1.]))]


From $A_{\Gamma}$ we can take $\sigma_{\Gamma}^{A_{\Gamma}}(\Gamma)$ by taking the union of $C_{\Gamma}(\sigma_{\Gamma}^{A_{\Gamma}}(T))$ for each $T$.

In [10]:
#Get break div for trees

lbm=[]

for data in M: 
    index=data[0]
    ass=data[1]
    vhead=head(n,index)
    vtail=tail_plus(n,index)

    lbm.append(ass+vhead)
    lbm.append(ass+vtail)

#check size:

lbm=np.unique(lbm,axis=0)

In [11]:
print(lbm)
# [[ 0.  1.  0.  0.]
#  [ 1.  0.  0.  0.]
#  [ 1.  0.  1. -1.]
#  [ 1.  1.  0. -1.]]

[[ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  1. -1.]
 [ 1.  1.  0. -1.]]
4
